In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import glob
import os
import math
from scipy.optimize import minimize

In [2]:
def cov_df_generator(df, correct_index):
    '''Transform cov_mat file into complete diagonal matrix'''
    
    df_temp = df.pivot_table(index='ROW_INDEX', columns='COLUMN_INDEX', values='VALUE', fill_value = 0)
    
    df_original = df_temp.copy()
    
    np.fill_diagonal(df_temp.values, 0)
    
    final_df = (df_original.T + df_temp).reindex(index = correct_index, columns = correct_index)
    
    return final_df

def minimum_weight_constraint(df):
    in_constraint = True
    
    for weight in df["WEIGHTS"][df["WEIGHTS"] > 0]:
        if weight < 0.001:
            in_constraint = False
            break
    
    return in_constraint

def num_of_portfolio_constraint(df):
    in_constraint = True
    
    if 50 <= np.count_nonzero(df["WEIGHTS"]) <= 70:
        pass
    else:
        in_constraint = False
        
    return in_constraint

def tracking_error_constraint(df, cov_df):
    in_constraint = True
    df_matrix = np.matrix(df["WEIGHT_DIFF"])
    cov_df_matrix = np.matrix(cov_df)
    
    traking_error = np.sqrt(df_matrix * cov_df_matrix * df_matrix.T)
    
    if (traking_error < 0.05) or (traking_error > 0.1):
        in_constraint = False
    
    return in_constraint

def weight_diff_constraint(df):
    in_constraint = True
    
    if (max(df["WEIGHT_DIFF"]) >= 0.05) or (min(df["WEIGHT_DIFF"]) <= -0.05):
        in_constraint = False
        
    return in_constraint

def mcapq_constraint(df):
    in_constraint = True
    
    for i in range(1, 6):
        if bool(-0.1 <= sum(df["WEIGHT_DIFF"][df["MCAP_Q"] ==  i ]) <= 0.1):
            pass
        else:
            in_constraint = False
            break
        
    return in_constraint


def sector_constraint(df):
    in_constraint = True
    
    for i in df["SECTOR"].unique():
        if bool(-0.1 <= sum(df["WEIGHT_DIFF"][df["SECTOR"] == i]) <= 0.1):
            pass
        else:
            in_constraint = False
            break
    return in_constraint

def active_share_constraint(df):
    in_constraint = True
    
    if bool(0.6 <= (1 - sum(np.minimum(df["WEIGHTS"], df["BENCH_WEIGHT"]))) <= 1.0):
        pass
    else:
        in_constraint = False
    
    return in_constraint

def beta_constraint(df):
    in_constraint = True
    
    if abs((df["WEIGHT_DIFF"] * df["BETA"]).sum()) > 0.1:
        in_constraint = False
    
    return in_constraint


def meet_constraints(df, cov_df):
    '''For all Constraints'''
    in_all_constraints = True
    
    if num_of_portfolio_constraint(df):
        if minimum_weight_constraint(df):
            if weight_diff_constraint(df):
                if beta_constraint(df):
                    if sector_constraint(df):
                        if mcapq_constraint(df):
                            if active_share_constraint(df):
                                if tracking_error_constraint(df, cov_df):
                                    pass
                                else:
                                    in_all_constraints = False
                            else:
                                in_all_constraints = False
                        else:
                            in_all_constraints = False
                    else:
                        in_all_constraints = False
                else:
                    in_all_constraints = False
            else:
                in_all_constraints = False
        else:
            in_all_constraints = False
    else:
        in_all_constraints = False
    
    return in_all_constraints

In [3]:
# Functions using in Scipy.Minimize
def objective(x):

    return (np.matrix(x - BENCH) * np.matrix(cov_df_temp) * np.matrix(x - BENCH).T) - \
           (LAMBDA * np.matrix(x - BENCH) * np.matrix(ALPHA).T)

def constraint4(x):
    SUM = 1
    return x.sum() - SUM

def constraint5(x):

    return (0.05 - abs(x - BENCH))

def constraint6(x):

    sector_list = []

    for sector in df_temp["SECTOR"].unique():
        sector_list.append(abs(sum(x[sector_iindex_dict[sector]] - df_temp["BENCH_WEIGHT"][sector_iindex_dict[sector]])))

    return 0.1 - np.array(sector_list)

def constraint7(x):
    mcapq_list = []

    for mcapq in range(1, 6):
        mcapq_list.append(abs(sum(x[mcapq_iindex_dict[mcapq]] - df_temp["BENCH_WEIGHT"][mcapq_iindex_dict[mcapq]])))

    return 0.1 - np.array(mcapq_list)

def constraint8(x):

    return 0.1 - abs(sum((x - BENCH) * df_temp["BETA"]))

def constraint10_1(x):
    return np.minimum(x, BENCH).sum()

def constraint10_2(x):
    return 0.4 - np.minimum(x, BENCH).sum()

def constraint11_1(x):
    return (0.1 - math.sqrt(np.matrix(x - BENCH) * np.matrix(cov_df_temp) * np.matrix(x - BENCH).T))

def constraint11_2(x):
    return math.sqrt((np.matrix(x - BENCH) * np.matrix(cov_df_temp) * np.matrix(x - BENCH).T)) - 0.05

con4 = {'type': 'eq', 'fun': constraint4}
con5 = {'type': 'ineq', 'fun': constraint5}
con6 = {'type': 'ineq', 'fun': constraint6}
con7 = {'type': 'ineq', 'fun': constraint7}
con8 = {'type': 'ineq', 'fun': constraint8}
con10_1 = {'type': 'ineq', 'fun': constraint10_1}
con10_2 = {'type': 'ineq', 'fun': constraint10_2}
con11_1 = {'type': 'ineq', 'fun': constraint11_1}
con11_2 = {'type': 'ineq', 'fun': constraint11_2}

# Import Data

In [4]:
start = time.time()

Time_Series_Data_path = "C:/Users/user/Desktop/INFORMS/Timeseries_data_SP500.txt"
opt_path = "C:/Users/user/Desktop/INFORMS/ResultsTemplate_Excel.xlsx"

TSD_df = pd.read_csv(Time_Series_Data_path, parse_dates= True, index_col= 'DATE', sep='\t')
template_df = pd.read_excel(opt_path, parse_dates=True, index_col='DATE', converters={'SEDOL': str})

#Use os.path.basename to extract the file name then extract only digit part to specify each dataframe

cov_mat_dict = {}
work_dict = {}

for file_path in glob.glob("C:/Users/user/Desktop/INFORMS/Riskmodels/cov_mat_*.csv"):
    date = os.path.basename(file_path).split('_')[2].replace('.csv','')
    
    work_dict[date] = pd.merge(TSD_df.loc[date], template_df.loc[date], how = 'outer', on = 'SEDOL').fillna(0)
    work_dict[date].set_index('SEDOL', inplace = True)
    
    correct_index = work_dict[date].index.values
    cov_mat_dict[date] = cov_df_generator(pd.read_csv(file_path), correct_index)

#missing value
cov_mat_dict['2016-07-06'].loc["BYT3MK1", "BYT3MK1"] = 0.5

cov_mat_dict['2016-07-06'].fillna(0, inplace = True)

end = time.time()
print(str(end - start) + ' Seconds')

69.62708330154419 Seconds


In [5]:
# check if there is any missing and 0 in any diagonal
for date in cov_mat_dict:
    if (cov_mat_dict[date].values.diagonal() == 0).any() or (cov_mat_dict[date].isnull().sum().sum() != 0):
        print(date)

In [8]:
work_dict['2007-01-31'].head()

,SECTOR,BETA,ALPHA_SCORE,NAME,BENCH_WEIGHT,MCAP_Q,WEIGHTS,FOUR_WEEKLY_RETURN
SEDOL,,,,,,,,
2038430,Financials,0.504252,-0.000006,Equity Office Properties Trust,0.001496,2,0.0,0.153207
2215460,Consumer Staples,0.548676,0.000010,"ConAgra Foods, Inc.",0.001004,2,0.0,-0.041111
2963899,Consumer Staples,1.002699,-0.000017,"Whole Foods Market, Inc.",0.000473,4,0.0,-0.075858
2767228,Industrials,0.965206,0.000006,"Rockwell Collins, Inc.",0.000874,2,0.0,0.077737
2464165,Materials,0.763931,0.000009,International Flavors & Fragrances Inc.,0.000290,5,0.0,-0.013832


In [10]:
# Covariance Matrix
cov_mat_dict['2007-01-31'].head()

ROW_INDEX,2038430,2215460,2963899,2767228,2464165,2947956,B8KQN82,2769503,2632650,2548616,...,2881537,2336747,2754060,2572303,2182779,2702791,2511328,2803014,2411138,2665861
COLUMN_INDEX,,,,,,,,,,,,,,,,,,,,,
2038430,0.077921,0.009070,0.009329,0.008081,0.011941,0.003488,0.0,0.012491,0.012665,0.012539,...,0.013662,0.011992,0.013563,0.011272,0.005086,0.009537,0.011000,0.010506,0.019549,0.015134
2215460,0.009070,0.067025,-0.000298,0.006914,0.007516,0.006680,0.0,0.007670,0.008092,0.008283,...,0.004217,0.005525,0.008284,0.006590,0.006392,0.006160,0.007255,0.005533,0.011529,0.007633
2963899,0.009329,-0.000298,0.170561,0.014857,-0.000337,0.008904,0.0,0.019160,0.021281,0.019849,...,0.039549,0.010187,0.023943,0.003363,0.000619,0.010350,0.009881,0.010569,0.029499,0.020015
2767228,0.008081,0.006914,0.014857,0.071527,0.011042,0.011978,0.0,0.016119,0.024658,0.020370,...,0.028976,0.008151,0.024680,0.008782,0.013888,0.010228,0.013067,0.010608,0.025425,0.022367
2464165,0.011941,0.007516,-0.000337,0.011042,0.058326,0.007445,0.0,0.009201,0.013731,0.012408,...,0.012174,0.006949,0.012786,0.010936,0.010290,0.007281,0.015725,0.007489,0.016248,0.013058


In [13]:
print("There are {} dates".format(len(work_dict)))

There are 131 dates


# Optimization

In [6]:
start = time.time()

# The dictionary for the optimal solutions of each date
optimal_solutions = {}

for date in work_dict:
    if (date == '2007-01-03'):
        continue
        
    print(date)
    
    df_temp = work_dict[date].copy()
    cov_df_temp = cov_mat_dict[date].copy()
    
    # For functions
    LAMBDA = 1
    BENCH = df_temp["BENCH_WEIGHT"]
    ALPHA = df_temp['ALPHA_SCORE']
    
    mcapq_iindex_dict = {}
    sector_iindex_dict = {}

    num_of_selection = 70

    while df_temp.groupby(["MCAP_Q", "SECTOR"]).size().multiply(num_of_selection / len(df_temp)).round().sum() > 70:
        num_of_selection -= 1

    type_num = df_temp.groupby(["MCAP_Q", "SECTOR"]).size().multiply(num_of_selection / len(df_temp)).round().to_dict()


    # Integer index of all sector type stocks
    for sector in df_temp["SECTOR"].unique():
        sector_iindex_dict[sector] = [df_temp.index.get_loc(i) for i in df_temp[df_temp["SECTOR"] == sector].index]

    # Integer index of all mcapq type stocks
    for mcapq in range(1, 6):
        mcapq_iindex_dict[mcapq] = [df_temp.index.get_loc(i) for i in df_temp[df_temp["MCAP_Q"] == mcapq].index]
    
    print(len(df_temp))
    
    signal = 0
    counter = 0
    
    while signal == 0:
        
        selected_index = set()
        
        for key, value in type_num.items():
            selected_index.update(set(np.random.choice(df_temp[(df_temp["MCAP_Q"] == key[0]) & (df_temp["SECTOR"] == key[1])].\
                                                       index.values, size = int(value), replace = False)))

        
        bnds = []

        for i in df_temp.index.values:
            if i in selected_index:
                bnds.append((0.001, 1))
            else:
                bnds.append((0, 0))


        cons = [con4, con5, con6, con7, con8, con10_1, con10_2, con11_1, con11_2]


        sol = minimize(objective, df_temp["WEIGHTS"], method='SLSQP', bounds=bnds, constraints=cons, options= {'disp': True})
        
        if sol.success:
            optimal_solutions[date] = sol.x
            signal = 1
        elif counter > 10:
            break
        else:
            counter += 1


end = time.time()
print(str(end - start) + " seconds")

2007-01-31
493
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.002498746295822637
            Iterations: 7
            Function evaluations: 3465
            Gradient evaluations: 7
2007-02-28
494
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.002500746822217841
            Iterations: 7
            Function evaluations: 3472
            Gradient evaluations: 7
2007-03-28
494
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.0025019345464651773
            Iterations: 7
            Function evaluations: 3472
            Gradient evaluations: 7
2007-04-25
494
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.0025026609440609617
            Iterations: 7
            Function evaluations: 3472
            Gradient evaluations: 7
2007-05-23
494
Optimization terminated successfully.    (Exit mode 0)
            Curr

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.00249883874493625
            Iterations: 7
            Function evaluations: 3493
            Gradient evaluations: 7
2009-02-25
497
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 0.026671799961822742
            Iterations: 1
            Function evaluations: 499
            Gradient evaluations: 1
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 0.026671799961822742
            Iterations: 1
            Function evaluations: 499
            Gradient evaluations: 1
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 0.026671799961822742
            Iterations: 1
            Function evaluations: 499
            Gradient evaluations: 1
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 0.026671799961822742
            Iterations: 1
       

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.002505506891729193
            Iterations: 8
            Function evaluations: 4000
            Gradient evaluations: 8
2011-02-23
498
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.002501288750311828
            Iterations: 7
            Function evaluations: 3500
            Gradient evaluations: 7
2011-03-23
498
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.002499943553992042
            Iterations: 7
            Function evaluations: 3500
            Gradient evaluations: 7
2011-04-20
498
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.00250157032023338
            Iterations: 8
            Function evaluations: 4001
            Gradient evaluations: 8
2011-05-18
498
Optimization terminated successfully.    (Exit mode 0)
            Current function value

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.0024995915631885212
            Iterations: 7
            Function evaluations: 3507
            Gradient evaluations: 7
2013-11-27
499
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.0024984318561635664
            Iterations: 7
            Function evaluations: 3508
            Gradient evaluations: 7
2013-12-25
499
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.0024973038763407163
            Iterations: 8
            Function evaluations: 4008
            Gradient evaluations: 8
2014-01-22
499
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.0025018958682051467
            Iterations: 7
            Function evaluations: 3507
            Gradient evaluations: 7
2014-02-19
499
Optimization terminated successfully.    (Exit mode 0)
            Current function 

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.002504451557932541
            Iterations: 7
            Function evaluations: 3549
            Gradient evaluations: 7
2016-08-31
505
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.0025041102160642667
            Iterations: 7
            Function evaluations: 3549
            Gradient evaluations: 7
2016-09-28
505
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.0025016764141361086
            Iterations: 6
            Function evaluations: 3043
            Gradient evaluations: 6
2016-10-26
505
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.0025023114444012347
            Iterations: 7
            Function evaluations: 3549
            Gradient evaluations: 7
2016-11-23
505
Optimization terminated successfully.    (Exit mode 0)
            Current function v